<a href="https://colab.research.google.com/github/ewilson2023/History-Name-Mashup-Generator/blob/main/HistoryChalengeNameGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is this
A "fantasy" name generator. Picks syllables of names from Ancient Egypt, Ancient Greece, Ancient Rome, and Ancient Near East and generates a mashup name
# How to use
if the section "Classes and Utilities" isn't minimised, minimize it.
1) First, under section "Classes and Utilities", press the play button to the left of "(number) of cells hidden".
2) Then, press the play button in the cell/block under the section "Main".

# Classes and Utilities

In [228]:
# class for each cultur, storing sorted dictionaries

class Culture:
  def __init__(self, p, m, s, name):
    self.name = name
    self.pre = sort(p)
    self.mid = sort(m)
    self.suf = sort(s)




*   dicts lack explicit structure
* TypedDict: a fixed-shape dictionary where the keys are always the same bucket names, and each key always maps to the same kind of value


   

In [229]:
from typing import TypedDict

# takes the form
# "name":   list["string"],
# "name":   list[int],
"""
SortBuckets is not a class at runtime. A SortedBuckets b
is a dictionary that has specific keys and value types.
b["all"] is a list of strings,
b["c_any"] is a list of ints
"""
class SortBuckets(TypedDict):
    all: list[str]
    vv: list[int]; cc: list[int]
    vc: list[int]; cv: list[int]
    c: list[int];  v: list[int]
    only_c: list[int]; only_v: list[int]
    c_any: list[int]; v_any: list[int]
    any_c: list[int]; any_v: list[int]

In [230]:
# return a dictionary sorted by consonant/vowel ending
def sort(list: list[str]) -> dict:
    dict = {
          "all" :[],
          "vv": [], "cc": [],
          "vc": [], "cv": [],
          "c": [],  "v": [],

          "only_c": [], "only_v": [],
          "c_any": [], "v_any": [],
          "any_c": [], "any_v": [],
      }
    for idx, str in enumerate(list):
        dict["all"].append(str)

        first = str[0]
        last = str[len(str) - 1]

        if len(str) == 1:
          if isConsonant(first) and first != "y":
            dict["c"].append(idx)
            dict["cc"].append(idx)
            dict["c_any"].append(idx)
            dict["any_c"].append(idx)
            dict["only_c"].append(idx)
          else:
            dict["v"].append(idx)
            dict["vv"].append(idx)
            dict["v_any"].append(idx)
            dict["any_v"].append(idx)
            dict["only_v"].append(idx)
          continue

        else:
          if (isConsonant(first) and first != "y") or (first == "y" and isVowel(str[1])):
            dict["c_any"].append(idx)

            if all_V_or_all_C(str) == "c":
              dict["only_c"].append(idx)

            if isConsonant(last) or (str[len(str)-2]+last) == "qu":
              dict["cc"].append(idx)
              dict["any_c"].append(idx)
            else:
              dict["cv"].append(idx)
              dict["any_v"].append(idx)
          else:
            dict["v_any"].append(idx)
            if all_V_or_all_C(str) == "v":
              dict["only_v"].append(idx)

            if isConsonant(last) or (str[len(str)-2]+last) == "qu":
              dict["vc"].append(idx)
              dict["any_c"].append(idx)
            else:
              dict["vv"].append(idx)
              dict["any_v"].append(idx)
          #    dict["only_v"].append(idx)


    return dict


In [231]:
# fix length of checked chars if adding other fordden ends len!=2
forbidden_ends: list[str] = ["qu", "dn"]

In [232]:
def isVowel(char: str) -> bool:
  char = char.lower()
  vowels = ["a", "e", "i", "o", "u", "ë"]

  if char in vowels:
    return True
  else:
    return False

def isConsonant(char: str) -> bool:
  return not isVowel(char)

# returns 1 for all vowels, 0 for all consonants, -1 else
def all_V_or_all_C(str):
  str = str.lower()
  vowels = ["a", "e", "i", "o", "u", "ë" ]
  allV: bool = True
  allC: bool = True

  if str[0] == "y" and str[1] in vowels:
    return ""

  for i, char in enumerate(str):
    if char.isalpha() == False:
      continue

    if char not in vowels:
      allV = False
    if char in vowels:
      allC = False

  if allV:
    return "v"
  elif allC:
    return "c"
  else:
    return ""


In [233]:
import random

""" Pick an index into b["all"], using fallback logic:
      1) try the preferred index-bucket (e.g., "c_any")
      2) if empty, try the other bucket (e.g., "v_any")
      3) if still empty, fall back to ANY valid index from b["all"]
    Returns: an int index suitable for b["all"][idx] """
def pick_idx( cult: Culture,
    b: SortBuckets, prefer: str, other: str) -> int:
      pool: list[int] = b[prefer]     # ex. b = [0, 5, 8] (idxs)
      if not pool:                    # if empty, pick from other list of idxs
          pool = b[other]             # still a list[int]
      if not pool:                    # if still empty, pick from all
          pool = list(range(len(b["all"])))
      return random.choice(pool)      # return random idx int from the pool

def pick_syllable(
    cult: Culture,
    next: str, part_choice: str ) -> str:
  sbl: str
  idx: int

  if part_choice == "prefix":
    word_part = cult.pre
  elif part_choice == "middle":
    word_part = cult.mid
  else: # part == "suffix"
    word_part = cult.suf

  while True:
    # choose what pool to get the index from
    if next == 'c':
      idx = pick_idx(cult, word_part, "c_any", "v_any")
    else: # next == 'v'
      idx = pick_idx(cult, word_part, "v_any", "c_any")

    # choose the syllable associated with this index
    sbl = word_part.get("all")[idx]
    break

  # if a prefix was chosen in fallback, make the
  # first letter lowercase (hypen aware)
  if part_choice != "prefix" and sbl[0].isalpha() and sbl[0].isupper():
    sbl[0] = sbl[0].lower()

  return sbl


def fix_end(cult: Culture, name: str):

  pool: list[int]
  new_suf: str

  while True:
    cult = random.choice(all_cultures)
    pool: list[int] = cult.suf["v_any"]
    # if empty, search again
    if not pool:
        continue

    # pick the syllable
    idx = random.choice(pool)
    new_suf = cult.suf.get("all")[idx]

    # if that makes the name too long, try again
    if len(name) > 7 and len(new_suf) > 2 :
      continue
    else:
      break


  cult_name: str = cult.name

  result = [ new_suf, cult_name]
  return result


In [234]:
import random

##################### v2
def generateName(cultures: list[Culture]) -> None:
  """############### PREFIX ###############"""
  name: str

  cult: Culture = random.choice(cultures)

  # prefix: random of all this cultures's prefixes
  p: str = random.choice(cult.pre.get("all"))

  debug1: str = (cult.name)
  debug2: str = (p)

  name = p;   """add it to name"""

  if isVowel(p[len(p) - 1]):
    next = 'c'
  else:
    next = 'v'

  """############### MIDDLE ###############"""
  m: str = ""

  if random.choice([True, False]):

    # choose a new culture
    cult = random.choice(cultures)

    m = pick_syllable(cult, next, "middle")

    if m != "":

      """add it to name"""
      name = name + m
      debug1 = (debug1+ " - " + cult.name)
      debug2 = (debug2+ " - " + m)

      # adjust for "qu" ending
      last2 = name[len(name) - 2] + name[len(name) - 1]
      if isConsonant(m[len(m) - 1]) or (last2 == "qu"):
        next = 'v'
      else:
        next = 'c'

    # 25% chance of second mid syllable, same culture
    if random.choice([True, False]) and len(name) < 7:

      m = pick_syllable(cult, next, "middle")

      if m != "":

        """add it to name"""
        name = name + m
        debug1 = (debug1+ " - " + cult.name)
        debug2 = (debug2+ " - " + m)

        # adjust for "qu" ending
        last2 = name[len(name) - 2] + name[len(name) - 1]
        if isConsonant(m[len(m) - 1]) or (last2 == "qu"):
          next = 'v'
        else:
          next = 'c'


  """############### SUFFIX ###############"""

  # choose a new culture
  cult = random.choice(cultures)

  s = pick_syllable(cult, next, "suffix")

  """add it to name"""
  name = name + s
  debug1 = (debug1+ " - " + cult.name)
  debug2 = (debug2+ " - " + s)

  if name[len(name) - 1]+name[len(name) - 2] in forbidden_ends:
    fix_end(cult, name)



  """############### OUTPUT ###############"""
  print(name)
  print("  "+debug1)
  print("  "+debug2 + "\n")




In [235]:
# list syllables for all cultures
#### EGYPTIAN ####
e_p = ["A","Nu", "Ha", "Dje", "Djo", "Amen", "Amon", "Anu", "Bas",
        "Sob", "Nef", "Nefer", "Sen", "Ahm", "Akhe", "Khu", "Kh",
        "Khn", "Kha", "Ram", "Thut", "Sme", "Set"]

e_m = ["ph", "phth", "ho", "huti", "nou", "shep", "kn",
       "de", "phr", "khm", "kh", "dj", "mh", "khk", "er", "utme",
       "hm", "khe", "nemh", "khn", "fer", "f", "pta", "djm", "fer",
       "auten", "nkhk", "utmose", "ankh", "ankha", "kham"]

e_s = ["are", "eru", "ys", "ek","esu", "et", "em", "at", "ten",
       "eb", "ut", "uti", "hotep", "ho", "tep", "are", "aten",
       "tari", "ti", "fre", "fre", "tem", "fra", "mose", "mon", "ket",
       "kh", "det", "tep", "th", "nut", "sis", "mheb", "nkh",
       "ankh", "ptah","met", "kmet", "djmet", "bek", "feru", "khamon",
       "sret", "sobek", "khufu", "psut", "auten"]

#### ROMAN ####
r_p  = ["Luc", "Val", "Mar", "Ovi", "Gai", "Hel", "Agrip",
        "C", "Tac", "Ta", "Ae", "Au", "Aure", "Cae", "Cl", "Fla",
        "Gl", "Gn", "Iu", "Ju", "Jul", "Liv", "Ma", "Marc",
        "Max", "Maxi", "Por", "Prisc", "Quint", "Sci", "Sev",
        "Silv", "Tit", "Va", "Vi", "Val", "Var"]
r_m  = ["an", "in", "ian", "ae", "an", "en", "ar", "qu", "v",
        "c", "n", "sc", "t","p", "aia", "l", "lian", "mil", "qui", "ll",
        "ss", "ssian", "ian", "aud", "au", "ci", "m", "iab", "ael",
        "up", "rc", "nl", "cell", "ae", "ver", "lp", "rqu", "quin",
        "nti", "nt", "ell", "ni"]
r_s  = ["ia","ius", "us", "ina", "ella", "a", "iana", "aeus"]

#### GREECE ####
g_p  = [ "Lys", "Kal", "Ari","Rho", "Kor", "Arte", "Ae", "Ch", "Chl",
        "Deme", "Dem", "Tel", "Tele", "Thou", "Eu", "A", "Tha", "Da",
        "Her", "Hesp", "Ny", "Ph", "Per", "Pel", "Phi", "Th", "Theo",
         "Tha", "Xa", "Xe", "Xeno", "Zo", "Ze", "Louk"]
g_m    = ["er", "in", "th", "t", "kr", "d", "tr", "k", "ll"
          "ph", "gor", "ch", "nt", "sch", "l", "ga", "pe", "ng", "ei"
          "el", "gath", "x", "xan", "lk", "ia", "mp", "el", "rk"
          "ari", "sth", "ian", "nth", "pho", "phe" "sthe", "hem", "rg"
          "thy", "pp", "mach", "ymph", "mph", "meni", "ych", "ou"]
g_s   = ["ius", "ias", "is", "as",  "ache", "one", "is", "des", "os"
         "ios", "us", "es", "eas", "eus", "ate", "ates", "ice", "sos"
         "os", "oë", "eus", "sia", "kles", "ion", "e", "ton", "eon",
         "phon", "es", "d", "od", "iod", "ene", "ates", "kles", "che"]

#### NEAR EAST ####
ne_p = ["Ha", "Hamm", "Has", "Nab", "Nabo", "Nabu", "Esar", "E",
        "Bel", "Sar", "Sha", "Shal", "En", "Enh",]
ne_m = ["mmu", "dru", "apla", "pol", "mmur", "nn", "zz", "mil",
        "sharra", "sha", "sh", "kk", "ira", "ban", "shur", "ch",
        "dn", "adn", "lm", "man", "a", "shl", "nhe", "dd", "rh"  ]
ne_s = ["ssar", "assar", "usur", "nna", "car", "ar", "zzar"
        "don", "rri", "i", "nezzar", "ton", "gon", "mis", "amis"
        "bi", "neser", "mat", "at", "ltum", "tum", "rra", "shua"]

# create class instance for each culture
Egypt = Culture(e_p, e_m, e_s, "Egyptian")
Rome = Culture(r_p, r_m, r_s, "Roman")
Greece = Culture(g_p, g_m, g_s, "Greek")
Near_East = Culture(ne_p, ne_m, ne_s, "Near East")

# list of all cultures/classes
all_cultures = [Egypt, Rome, Greece, Near_East]


# Main
AFTER pressing play on "Classes and Utilities", press the play button below.

In [223]:
# <- Hover over me!! Click that button!!

for i in range(5):
  generateName(all_cultures)

Khudem
  Egyptian - Greek - Egyptian
  Khu - d - em

Helar
  Roman - Near East
  Hel - ar

Amanat
  Egyptian - Near East - Near East
  A - man - at

Ramerautena
  Egyptian - Egyptian - Egyptian - Roman
  Ram - er - auten - a

Sevaeus
  Roman - Roman
  Sev - aeus



# hide
dw abt this

In [236]:
# debug: list all of type
def print_list(c, word_part, type: str):
  idx_list = word_part[type]
  print(c.name)
  print("\b ("+str(len(idx_list))+")" )

  all: list[str] = word_part["all"]

  if idx_list == []:
    print("  []\n")
    return

  print("  [ ")

  if type == "all":
    for i in idx_list:
      print ("\b"+ i + ", ")
  else:
    for i in idx_list:
      print ("\b"+ all[i]+ ", ")

  print("\b\b\b ]\n" )


for c in all_cultures:
  test_this = "cv"
  print_list(c, c.mid, test_this)



Egyptian
 (6)
  [ 
ho, 
huti, 
nou, 
de, 
khe, 
pta, 
 ]

Roman
 (4)
  [ 
qui, 
ci, 
nti, 
ni, 
 ]

Greek
 (6)
  [ 
ga, 
pe, 
rkari, 
pho, 
phesthe, 
meni, 
 ]

Near East
 (5)
  [ 
mmu, 
dru, 
sharra, 
sha, 
nhe, 
 ]



In [227]:
cult = Egypt
tester = cult.suf
arr = tester.get("vc")
all = tester.get("all")


print(cult.name)
print(all)
print(arr)
print()

result=""

for i, idx in enumerate(arr):
  result = result + ", "+ all[idx]

result = result.removeprefix(", ")
print(result)


Egyptian
['are', 'eru', 'ys', 'ek', 'esu', 'et', 'em', 'at', 'ten', 'eb', 'ut', 'uti', 'hotep', 'ho', 'tep', 'are', 'aten', 'tari', 'ti', 'fre', 'fre', 'tem', 'fra', 'mose', 'mon', 'ket', 'kh', 'det', 'tep', 'th', 'nut', 'sis', 'mheb', 'nkh', 'ankh', 'ptah', 'met', 'kmet', 'djmet', 'bek', 'feru', 'khamon', 'sret', 'sobek', 'khufu', 'psut', 'auten']
[2, 3, 5, 6, 7, 9, 10, 16, 34, 46]

ys, ek, et, em, at, eb, ut, aten, ankh, auten
